## Scenario 2: cross-functional team with one data scientist working on an ML model
- Tracking server: yes, local server
- Backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")

2022/06/01 11:25:23 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
c:\Users\adamgifford_behavr.DESKTOP-PQU0D8M\anaconda3\envs\mlopszoom\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [4]:
print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: './artifacts_local/1/d93148a6944e41b4aa79fef7aff84fd6/artifacts'


In [5]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts_local/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [7]:
client.list_registered_models()

[]

In [11]:
# not sure why there are 2 runs listed, even though i only ran a single run and there is
# no folder corresponding th run_id 'd93148a6944e41b4aa79fef7aff84fd6'
run_id = client.list_run_infos(experiment_id='1')[1].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2022/06/01 11:36:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 2
Created version '2' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1654097811689, current_stage='None', description='', last_updated_timestamp=1654097811689, name='iris-classifier', run_id='98052f192e984b768944dd9a765f946b', run_link='', source='./artifacts_local/1/98052f192e984b768944dd9a765f946b/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>